In [1]:
import xml.etree.ElementTree as ET
from dataclasses import dataclass
from math import sqrt

# Parsing raw file

In [2]:
tree = ET.parse('res/n4-all-raw.svg')
root = tree.getroot()

In [3]:
def tagFromElement(e):
    return e.tag.split('}')[-1]

In [4]:
def getFloat(element, tag):
    return float(element.attrib[tag].replace('mm', ''))

def getString(element, tag):
    return element.attrib[tag]

In [5]:
# https://developer.mozilla.org/en-US/docs/Web/SVG/Reference/Attribute/transform
def applyTransform(matrix, circle):
    x, y, r = circle
    [a, b, c, d, e, f] = matrix
    scale_x = sqrt(a*a + c*c)
    scale_y = sqrt(b*b + d*d)
    assert abs(scale_x - scale_y) < 0.01
    return (a*x + c*y + e, b*x + d*y + f, scale_x * r)

In [6]:
def groupToCircles(groupElement):
    transformMatrix = [1, 0, 0, 1, 0, 0]
    if 'transform' in groupElement.attrib:
        transform = getString(groupElement, 'transform')
        if 'matrix' in transform:
            transform = transform[transform.index('(') + 1 : transform.index(')')]
            transformMatrix = [float(v) for v in transform.split(',')]
        elif 'translate' in transform:
            transform = transform[transform.index('(') + 1 : transform.index(')')]
            transformMatrix = [1, 0, 0, 1] + [float(v) for v in transform.split(',')]
        
    circles = []
    for child in groupElement:
        tag = tagFromElement(child)
        circle = None
        if tag == 'circle':
            circle = (getFloat(child, 'cx'), getFloat(child, 'cy'), getFloat(child, 'r'))
        elif tag == 'ellipse':
            assert abs(getFloat(child, 'rx') - getFloat(child, 'ry')) < 0.01
            circle = (getFloat(child, 'cx'), getFloat(child, 'cy'), getFloat(child, 'rx'))
        
        if circle:
            circles.append(applyTransform(transformMatrix, circle))
            
    return circles

In [7]:
allGroups = []
for child in root:
    group = groupToCircles(child)
    if len(group) > 0:
        allGroups.append(group)
len(allGroups)

173

In [8]:
scale = 3
padding = 20

transformMatrix = [scale, 0, 0, scale, padding, padding]

for g in range(len(allGroups)):
    for c in range(len(allGroups[g])):
        allGroups[g][c] = applyTransform(transformMatrix, allGroups[g][c])

In [9]:
def getSize(circleGroups):
    xRange = [-1, -1]
    yRange = [-1, -1]
    for g in allGroups:
        for x, y, r in g:
            if xRange[0] == -1 or x - r< xRange[0]:
                xRange[0] = x - r
            if xRange[1] == -1 or x + r > xRange[1]:
                xRange[1] = x + r
            if yRange[0] == -1 or y - r < yRange[0]:
                yRange[0] = y - r
            if yRange[1] == -1 or y + r > yRange[1]:
                yRange[1] = y + r
    return (xRange[1] - xRange[0], yRange[1] - yRange[0])

# Writing to output
Colours and style sourced from original image hosted at https://oeis.org/A250001/a250001.svg

In [10]:
outputPath = "res/n4-all.svg"
width, height = getSize(allGroups)
width += 2 * padding
height += 2 * padding

with open(outputPath, 'w') as f:
    f.write(f'<svg width="{width}" height="{height}" xmlns="http://www.w3.org/2000/svg" version="1.2" xmlns:xlink="http://www.w3.org/1999/xlink">\n')
    f.write('\n')
    f.write('<!-- avg bsr 9.7 -->\n')
    f.write('\n')
    f.write('<defs>\n')
    f.write('  <linearGradient id="backColor">\n')
    f.write('    <stop offset="0%" stop-color="#9EC4BB"/>\n')
    f.write('  </linearGradient>\n')
    f.write('  <linearGradient id="color1">\n')
    f.write('    <stop offset="0%" stop-color="#2D2E2C"/>\n')
    f.write('  </linearGradient>\n')
    f.write('  <linearGradient id="color2">\n')
    f.write('    <stop offset="0%" stop-color="#2D2E2C"/>\n')
    f.write('  </linearGradient>\n')
    f.write('  <linearGradient id="color3">\n')
    f.write('    <stop offset="0%" stop-color="#2D2E2C"/>\n')
    f.write('  </linearGradient>\n')
    f.write('  <linearGradient id="color4">\n')
    f.write('    <stop offset="0%" stop-color="#2D2E2C"/>\n')
    f.write('  </linearGradient>\n')
    f.write('  <linearGradient id="bloomColor">\n')
    f.write('    <stop offset="0%" stop-color="#F7DECE"/>\n')
    f.write('  </linearGradient>\n')
    f.write('  <linearGradient id="stroke_color">\n')
    f.write('    <stop offset="0%" stop-color="#2D2E2C"/>\n')
    f.write('  </linearGradient>\n')
    f.write('\n')
    f.write(f'  <filter id="theBlur" filterUnits="userSpaceOnUse" x="0" y="0" width="{width}" height="{height}">\n')
    f.write('    <feGaussianBlur in="SourceGraphic" stdDeviation="45"/>\n')
    f.write('  </filter>\n')
    f.write('\n')
    f.write(' <g id="bright" style="fill:url(#bloomColor); fill-opacity:0.7">\n')
    for group in allGroups:
        for circle in group:
            f.write(f' <circle cx="{circle[0]}" cy="{circle[1]}" r="{circle[2]}"/>\n')
    f.write(' </g>\n')
    f.write('\n')
    f.write(' <g id="cover" style="fill:url(#backColor); fill-opacity:0.85">\n')
    for group in allGroups:
        for circle in group:
            f.write(f' <circle cx="{circle[0]}" cy="{circle[1]}" r="{circle[2]}"/>\n')
    f.write(' </g>\n')
    f.write('\n')
    f.write(' <g id="ccfg_fill" style="fill-opacity:0.05">\n')
    for group in allGroups:
        for circle in group:
            f.write(f' <circle cx="{circle[0]}" cy="{circle[1]}" r="{circle[2]}" fill="url(#color1)"/>\n')
    f.write(' </g>\n')
    f.write('\n')
    f.write(' <g id="ccfg_stroke" style="fill:none;stroke:0.5">\n')
    for group in allGroups:
        for circle in group:
            f.write(f' <circle cx="{circle[0]}" cy="{circle[1]}" r="{circle[2]}" stroke="url(#stroke_color)"/>\n')
    f.write(' </g>\n')
    f.write('\n')
    f.write('</defs>\n')
    f.write('\n')
    f.write(f'<rect x="0" y="0" width="{width}" height="{height}" style="fill:url(#backColor)"/>\n')
    f.write('<g>\n')
    f.write('\n')
    f.write('  <use xlink:href="#bright" filter="url(#theBlur)"/>\n')
    f.write('  <use xlink:href="#cover"/>\n')
    f.write('  <use xlink:href="#ccfg_fill"/>\n')
    f.write('  <use xlink:href="#ccfg_stroke"/>\n')
    f.write('</g>\n')
    f.write('\n')
    f.write('</svg>\n')

    

In [11]:
csvOutputPath = 'res/n4-all.csv'

with open(csvOutputPath, 'w') as f:
    for group in allGroups:
        for x, y, r in group:
            f.write(f"{x},{y},{r}\n")